In [1]:
import warnings
warnings.filterwarnings( 'ignore' )
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, TimeSeriesSplit
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.impute import SimpleImputer

In [2]:
trainpath = '../../CleanPartitions/trainp2.csv'
testpath = '../../CleanPartitions/testp2.csv'

traindata = pd.read_csv(trainpath, encoding='ISO-8859-1')
testdata = pd.read_csv(testpath, encoding='ISO-8859-1')

if 'attack_date' in traindata.columns:
    traindata = traindata.drop(columns=['attack_date'])

if 'attack_date' in testdata.columns:
    testdata = testdata.drop(columns=['attack_date'])

    print(f'shape train data: ', traindata.shape)
    print(f'shape test data: ', testdata.shape)

shape train data:  (6647, 25)
shape test data:  (2847, 25)


In [3]:
def split_data(dftrain, dftest):
    Ytrain = dftrain['gname']
    Xtrain = dftrain.drop(columns=['gname'])
    Ytest = dftest['gname']
    Xtest = dftest.drop(columns=['gname'])
    return Xtrain, Ytrain, Xtest, Ytest

def find_best_rfc(Xtrain, Ytrain):
     params = {
          'criterion': ["gini", "entropy"],
          'n_estimators': [5, 10, 20, 50, 100, 150, 200, 300, 500],
          'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
          'max_features': ['sqrt', 'log2']
          }

     rfc = RandomForestClassifier(random_state=42)

     tscv = TimeSeriesSplit(n_splits=5)


     grid_search =GridSearchCV(estimator=rfc, param_grid=params, cv = tscv)

     grid_search.fit(Xtrain, Ytrain)
     best_rfc = grid_search.best_estimator_
     #print(best_dt)
     return best_rfc

In [4]:
Xtrain, Ytrain, Xtest, Ytest = split_data(traindata, testdata)
best_rfc = find_best_rfc(Xtrain, Ytrain)
y_pred_rfc = best_rfc.predict(Xtest)
accuracy_gbc = accuracy_score(Ytest, y_pred_rfc)
print(f"Accuracy: {accuracy_gbc * 100:.2f}%")

Accuracy: 99.82%
